In [80]:
#Model with infinite battery capacity
#-----Data-----#
#Electricity demand (MWH). Index corresponds to hour 
demand = [43, 40, 36, 36, 35, 38, 41, 46, 49, 48, 47, 47, 
          48, 46, 45, 47, 50, 63, 75, 75, 72, 66, 57, 50]

purchaseLimit = 75         #Maximum power can be purchased

#Cost schedule
cost1 = 100
cost2 = 400
cutOff = 50               #Cut off point between first and second cost level

batCap = 30               #Battery power capacity

#Calculate daily cost without battery
totalCostWOBat = 0

for i = 1:24
    if(demand[i] <= cutOff)
        totalCostWOBat += demand[i]*cost1
    else
        totalCostWOBat += cutOff*cost1 + (demand[i] - 50)*cost2
    end
end

In [98]:
#-----Model-----#
using JuMP
m = Model()

@defVar(m, qBuy1[1:24] >= 0)          #Electricity bounght on the first cost level
@defVar(m, qBuy2[1:24] >= 0)          #Electricity bounght on the second cost level
@defVar(m, batLevel[1:25] >= 0)       #Battery level at each hour

@defExpr(totalCost, sum{qBuy1[i]*cost1 + qBuy2[i]*cost2, i in 1:24}) #Total cost

@addConstraint(m, batLevel[1] == 0)                                 #Initial battery level
#@addConstraint(m, batLevelLimit[i in 1:24], batLevel[i] <= batCap)  #Battery level must not exceed its capacity
@addConstraint(m, priceLevel[i in 1:24], qBuy1[i] <= cutOff)        #Cut off for the first price level
#Electricty bought must be within the limit
@addConstraint(m, avoidBlackout[i in 1:24], qBuy1[i] + qBuy2[i] <= purchaseLimit)   
#Flow constraint
@addConstraint(m, balance[i in 1:24], qBuy1[i] + qBuy2[i] + batLevel[i] == demand[i] + batLevel[i + 1])

@setObjective(m, Min, totalCost)    #Minimize cost

solve(m)
println("Total cost without battery: ", totalCostWOBat, " dollars")
println("Total cost with battery: ", getValue(totalCost), " dollars")
println("Savings: ", totalCostWOBat - getValue(totalCost), " dollars")
println("Battery levels: ", getValue(batLevel'))

Total cost without battery: 152400 dollars
Total cost with battery: 120000.0 dollars
Savings: 32400.0 dollars
Battery levels: [0.0 7.0 17.0 31.0 45.0 60.0 72.0 81.0 85.0 86.0 88.0 91.0 94.0 96.0 100.0 105.0 108.0 108.0 95.0 70.0 45.0 23.0 7.0 0.0 0.0]
